# 1. Import and combine all excel files

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import os
from pathlib import Path
import warnings

In [2]:
warnings.simplefilter("ignore")

In [3]:
# Get all excel files in data folder:
files = Path(os.getcwd()+'\\Querry Slot Data').rglob('*.xlsx')

In [4]:
file_list = [str(i) for i in files]

In [5]:
file_name = os.listdir(os.getcwd()+'\\Querry Slot Data')

In [8]:
# Loop through all the excel files and merge them into one:
df = pd.DataFrame()
for i in range(len(file_list)):
    df_temp = pd.read_excel(file_list[i])
    # Get additional info from file name:
    df_temp['file_name'] = file_name[i]
    df_temp['file_name'].replace('.xlsx','', inplace = True, regex = True)
    info = df_temp['file_name'].str.split('_', expand=True)
    info.columns = ['Reportdate','Season','Airport','Slot_type','Network']
    df_temp = pd.concat([df_temp,info],axis=1)
    
    # Drop unnecessary columns:
    df_temp.drop(['TIME.1','file_name'], axis=1, inplace = True, errors = 'ignore')
    # Melt slot date columns before merging:
    df_temp = pd.melt(df_temp, id_vars=['TIME','Reportdate', 'Season', 'Airport', 'Slot_type', 'Network'],
        value_vars=df_temp.columns.difference(['TIME','Reportdate', 'Season', 'Airport', 'Slot_type', 'Network']),
        var_name='Slot Date', value_name='No. Slot')
    df = pd.concat([df_temp,df], axis = 0)

In [9]:
df['day'] = df['Slot Date'].astype('str').apply(lambda x: x.split('-')[0])
df['month'] = df['Slot Date'].astype('str').apply(lambda x: x.split('-')[1])

In [10]:
df['season2'] = df['Season'].apply(lambda x: x[0])

In [12]:
df['year'] = df['Season'].astype('str').str.extract('(\d+)')

In [13]:
df['year'] = df.apply(lambda x: int('20'+x['year'])+1 
                      if (x['season2'] == 'W') and (x['month'] in (['Jan','Feb','Mar'])) 
                      else int('20'+x['year']), axis=1)

In [15]:
df['Reportdate'] = pd.to_datetime(df['Reportdate'], format='%Y%m%d')

In [18]:
d = {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7, 'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12}
df['month'] = df['month'].map(d)

In [19]:
df['Slot Date'] = pd.to_datetime(df[['year', 'month', 'day']])

In [21]:
result = df[['TIME','Reportdate','Season','Airport','Slot_type','Network','Slot Date','No. Slot']]

In [26]:
writer = pd.ExcelWriter("result.xlsx",  datetime_format='YYYY-MM-DD')

result.to_excel(writer,"Sheet1", index=False)
writer.close()